In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
import stk
import stko


def robust_energy(smiles, factories=[], max_tries=50):
    """
    시도 순서:
    1. stk.BuildingBlock + stko.MMFF/UFF
    2. RDKit로 직접 임베딩 + MMFF/UFF 수동 계산
    """
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)


    for seed in range(max_tries):
        try:
            bb = stk.BuildingBlock(
                smiles=smiles,
                functional_groups=factories,
                position_matrix=None
            )
            mmff = stko.MMFFEnergy()
            uff = stko.UFFEnergy()
            return mmff.get_results(bb).get_energy(), uff.get_results(bb).get_energy(), "stk"
        except RuntimeError as e:
            if f"seed value of {seed}" in str(e):
                continue
            else:
                break

    
    for seed in range(max_tries):
        mol_copy = Chem.Mol(mol)
        if AllChem.EmbedMolecule(mol_copy, randomSeed=seed) != 0:
            continue
        try:
            
            if AllChem.MMFFHasAllMoleculeParams(mol_copy):
                props = AllChem.MMFFGetMoleculeProperties(mol_copy)
                ff = AllChem.MMFFGetMoleculeForceField(mol_copy, props)
                mmff_energy = ff.CalcEnergy()
            else:
                mmff_energy = None

            
            if AllChem.UFFHasAllMoleculeParams(mol_copy):
                uff = AllChem.UFFGetMoleculeForceField(mol_copy)
                uff_energy = uff.CalcEnergy()
            else:
                uff_energy = None

            return mmff_energy, uff_energy, "rdkit"
        except Exception:
            continue

    return None, None, "failed"


smiles_df = pd.read_csv('smiles.csv')

descriptors_list = []

for idx in range(len(smiles_df['SMILES'])):
    smiles = smiles_df['SMILES'][idx]
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        print(f"[MolFromSmiles 실패] idx={idx}, SMILES={smiles}")
        continue

    mol = Chem.AddHs(mol, addCoords=True)
    row_dic = {}

    
    for name, fn in Descriptors._descList:
        row_dic[name] = fn(mol)

    
    try:
        AllChem.ComputeGasteigerCharges(mol)
        charges = [mol.GetAtomWithIdx(i).GetDoubleProp('_GasteigerCharge') for i in range(mol.GetNumAtoms())]
        row_dic['GasteigerChargeMax'] = max(charges)
        row_dic['GasteigerChargeMin'] = min(charges)
        row_dic['GasteigerChargeStd'] = np.std(charges)
    except Exception as e:
        row_dic['GasteigerChargeMax'] = None
        row_dic['GasteigerChargeMin'] = None
        row_dic['GasteigerChargeStd'] = None
        print(f"[Gasteiger 실패] idx={idx}, SMILES={smiles}\n{e}")

    
    has_Br = any(atom.GetSymbol() == 'Br' for atom in mol.GetAtoms())
    has_I  = any(atom.GetSymbol() == 'I' for atom in mol.GetAtoms())
    factories = []
    if has_Br:
        factories.append(stk.BromoFactory())
    if has_I:
        factories.append(stk.IodoFactory())

    
    mmff_energy, uff_energy, method = robust_energy(smiles, factories)
    row_dic['MMFFenergy'] = mmff_energy
    row_dic['UFFenergy'] = uff_energy
    row_dic['EnergyMethod'] = method

    if method == "failed":
        print(f"[에너지 완전 실패] idx={idx}, SMILES={smiles}")

    descriptors_list.append(row_dic)


descriptors_df = pd.DataFrame(descriptors_list)
descriptors_df = descriptors_df.loc[:, (descriptors_df != 0).any(axis=0)]
full_data = pd.concat([smiles_df.reset_index(drop=True), descriptors_df.reset_index(drop=True)], axis=1)


full_data.to_csv('descriptors.csv', index=False)
